# Training of AlexNet

In [1]:
import os
import numpy as np
import tensorflow as tf
import tf_keras as keras
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt


In [2]:
# parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 6

DATA_DIR = "../dataset_grayscale"
CLASSES = ["Normal", "Lung Opacity"]


In [3]:
images = []
labels = []

for label, cls in enumerate(CLASSES):
    class_dir = os.path.join(DATA_DIR, cls)
    for file in os.listdir(class_dir):
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            path = os.path.join(class_dir, file)
            img = Image.open(path)
            img = img.resize((IMG_SIZE, IMG_SIZE))
            img = np.array(img) / 255.0
            img = np.expand_dims(img, axis=-1)  # canal unique
            images.append(img)
            labels.append(label)


In [4]:
images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

In [5]:
indices = np.arange(len(images))
np.random.seed(42)
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

# split
total = len(images)
train_size = int(0.7 * total)
val_size   = int(0.15 * total)

# train, val, test
x_train = images[:train_size]
y_train = labels[:train_size]
x_val   = images[train_size:train_size+val_size]
y_val   = labels[train_size:train_size+val_size]
x_test  = images[train_size+val_size:]
y_test  = labels[train_size+val_size:]

print("Train:", x_train.shape)
print("Val:", x_val.shape)
print("Test:", x_test.shape)

# datasets
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Train: (1662, 224, 224, 1)
Val: (356, 224, 224, 1)
Test: (357, 224, 224, 1)


In [6]:

# ALEXNET ARCHITECTURE
model = keras.Sequential([
    keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1)),

    # Conv 1
    keras.layers.Conv2D(96, 11, strides=4, activation="relu"),
    keras.layers.MaxPooling2D(3, strides=2),

    # Conv 2
    keras.layers.Conv2D(256, 5, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(3, strides=2),

    # Conv 3
    keras.layers.Conv2D(384, 3, padding="same", activation="relu"),

    # Conv 4
    keras.layers.Conv2D(384, 3, padding="same", activation="relu"),

    # Conv 5
    keras.layers.Conv2D(256, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(3, strides=2),

    # Connected Layers
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation="relu"),
    keras.layers.Dropout(0.5),

    # Output
    keras.layers.Dense(1, activation="sigmoid")
])



In [7]:
# compilation
optimizer = keras.optimizers.legacy.Adam()
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy", keras.metrics.AUC(name="auc")]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        11712     
                                                                 
 max_pooling2d (MaxPooling2  (None, 26, 26, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 384)       1

In [8]:
# training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/6
52/52 [==============================] - 59s 1s/step - loss: 0.6997 - accuracy: 0.5187 - auc: 0.5191 - val_loss: 0.6915 - val_accuracy: 0.5309 - val_auc: 0.6046
Epoch 2/6
52/52 [==============================] - 52s 998ms/step - loss: 0.6645 - accuracy: 0.5728 - auc: 0.6143 - val_loss: 0.5876 - val_accuracy: 0.7022 - val_auc: 0.8540
Epoch 3/6
52/52 [==============================] - 52s 1s/step - loss: 0.4888 - accuracy: 0.7804 - auc: 0.8483 - val_loss: 0.4507 - val_accuracy: 0.7921 - val_auc: 0.9052
Epoch 4/6
52/52 [==============================] - 52s 1s/step - loss: 0.4075 - accuracy: 0.8201 - auc: 0.8950 - val_loss: 0.3843 - val_accuracy: 0.8230 - val_auc: 0.9083
Epoch 5/6
52/52 [==============================] - 51s 985ms/step - loss: 0.3980 - accuracy: 0.8255 - auc: 0.9004 - val_loss: 0.3970 - val_accuracy: 0.8062 - val_auc: 0.9051
Epoch 6/6
52/52 [==============================] - 53s 1s/step - loss: 0.3908 - accuracy: 0.8321 - auc: 0.9052 - val_loss: 0.3903 - val_acc

In [9]:
#evaluation
loss, acc, auc = model.evaluate(test_ds)
print("Test Accuracy:", acc)
print("Test AUC:", auc)

12/12 [==============================] - 3s 255ms/step - loss: 0.3555 - accuracy: 0.8487 - auc: 0.9198
Test Accuracy: 0.848739504814148
Test AUC: 0.9198367595672607


In [10]:
# resukts
y_true = []
y_pred = []

for x, y in test_ds:
    preds = model.predict(x)
    preds = (preds > 0.5).astype(int)
    y_true.extend(y.numpy())
    y_pred.extend(preds.flatten())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=CLASSES))

1/1 [==============================] - 0s 103ms/step
[[151  29]
 [ 25 152]]
              precision    recall  f1-score   support

      Normal       0.86      0.84      0.85       180
Lung Opacity       0.84      0.86      0.85       177

    accuracy                           0.85       357
   macro avg       0.85      0.85      0.85       357
weighted avg       0.85      0.85      0.85       357



2026-01-08 18:43:57.155812: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
# save
script_dir = os.getcwd() 
model.save(os.path.join(script_dir, '..', '..', 'models', 'my_alexnet.h5'))

/Users/tiago/Documents/esilvA5/ecole/ExplainAI/finprojet/Xai-unified-interface/venv/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Densenet

In [17]:
images = []
labels = []

for label, cls in enumerate(CLASSES):
    class_dir = os.path.join(DATA_DIR, cls)
    for file in os.listdir(class_dir):
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            path = os.path.join(class_dir, file)
            img = Image.open(path).resize((IMG_SIZE, IMG_SIZE))
            img = np.array(img) / 255.0
            if img.ndim == 2:  # grayscale -> RGB
                img = np.stack([img]*3, axis=-1)
            images.append(img)
            labels.append(label)

images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)


In [18]:
indices = np.arange(len(images))
np.random.seed(42)
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

# split
total = len(images)
train_size = int(0.7 * total)
val_size   = int(0.15 * total)

# train, val, test
x_train = images[:train_size]
y_train = labels[:train_size]
x_val   = images[train_size:train_size+val_size]
y_val   = labels[train_size:train_size+val_size]
x_test  = images[train_size+val_size:]
y_test  = labels[train_size+val_size:]

print("Train:", x_train.shape)
print("Val:", x_val.shape)
print("Test:", x_test.shape)

# datasets
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Train: (1662, 224, 224, 3)
Val: (356, 224, 224, 3)
Test: (357, 224, 224, 3)


In [19]:
base_model = keras.applications.DenseNet121(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])

In [20]:
# compilation
optimizer = keras.optimizers.legacy.Adam()
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy", keras.metrics.AUC(name="auc")]
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_1  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 256)               262400    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 7300161 (27.85 MB)
Trainable params: 262657 (1.00 MB)
Non-trainable params: 7037504 (26.85 MB)
___________

In [21]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/6
52/52 [==============================] - 69s 1s/step - loss: 0.4627 - accuracy: 0.7918 - auc: 0.8687 - val_loss: 0.3569 - val_accuracy: 0.8483 - val_auc: 0.9238
Epoch 2/6
52/52 [==============================] - 78s 2s/step - loss: 0.3481 - accuracy: 0.8562 - auc: 0.9231 - val_loss: 0.3400 - val_accuracy: 0.8567 - val_auc: 0.9256
Epoch 3/6
52/52 [==============================] - 79s 2s/step - loss: 0.3258 - accuracy: 0.8694 - auc: 0.9327 - val_loss: 0.3189 - val_accuracy: 0.8596 - val_auc: 0.9350
Epoch 4/6
52/52 [==============================] - 79s 2s/step - loss: 0.3019 - accuracy: 0.8857 - auc: 0.9431 - val_loss: 0.3137 - val_accuracy: 0.8596 - val_auc: 0.9367
Epoch 5/6
52/52 [==============================] - 84s 2s/step - loss: 0.3027 - accuracy: 0.8767 - auc: 0.9414 - val_loss: 0.3144 - val_accuracy: 0.8567 - val_auc: 0.9369
Epoch 6/6
52/52 [==============================] - 85s 2s/step - loss: 0.2861 - accuracy: 0.8833 - auc: 0.9488 - val_loss: 0.3073 - val_accuracy:

In [22]:
loss, acc, auc = model.evaluate(test_ds)
print("Test Accuracy:", acc)
print("Test AUC:", auc)

12/12 [==============================] - 16s 1s/step - loss: 0.2758 - accuracy: 0.8824 - auc: 0.9534
Test Accuracy: 0.8823529481887817
Test AUC: 0.953405499458313


In [23]:
y_true = []
y_pred = []

for x, y in test_ds:
    preds = model.predict(x)
    preds = (preds > 0.5).astype(int)
    y_true.extend(y.numpy())
    y_pred.extend(preds.flatten())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=CLASSES))

1/1 [==============================] - 1s 1s/step
[[163  17]
 [ 25 152]]
              precision    recall  f1-score   support

      Normal       0.87      0.91      0.89       180
Lung Opacity       0.90      0.86      0.88       177

    accuracy                           0.88       357
   macro avg       0.88      0.88      0.88       357
weighted avg       0.88      0.88      0.88       357



2026-01-08 18:58:24.222396: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
model.save(os.path.join(script_dir, '..', '..', 'models', 'my_densenet.h5'))

/Users/tiago/Documents/esilvA5/ecole/ExplainAI/finprojet/Xai-unified-interface/venv/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
